In [ ]:
import glob, os, pickle, datetime
import xgrads
import xarray as xa
from matplotlib import pyplot as plt
import numpy as np
from dask.distributed import Client

def fixTime(ctl):
    dt2000secs=round((datetime.datetime(2000,1,1)-datetime.datetime(1,1,1)).total_seconds())    
    ctl.tdef.samples=ctl.tdef.samples+dt2000secs
    return(ctl)

def getTimes(pathname):
    ctl = xgrads.CtlDescriptor(file=pathname)
    ctl=fixTime(ctl)
    return (ctl.tdef.samples)

def getCtlList(varname):
    pathlist=glob.glob('ape_2/cntl/??????????/' + varname + '.ctl',recursive=True)
    #build list of ctl objects for readin grads data
    ctlListt2000=[fixTime(xgrads.CtlDescriptor(file=path)) for path in pathlist]
    return(ctlListt2000)

def getVarTimeMean(varname):
    pathlist=glob.glob('interm/' + varname + '/*' + varname + '.tmean.nc',recursive=True)
    #need to add time coord to tmean files (length 1) to permit multifile open along time axis
    def preprocessing(ds): 
        return(ds.expand_dims(dim={'t':[np.datetime64(ds.attrs['t0'])]}))
    ds = xa.open_mfdataset(pathlist, concat_dim='t',combine='by_coords', preprocess = preprocessing)
    ds_tmean=ds.mean(dim='t').compute()
    return ds_tmean[varname]

In [ ]:
client = Client()
client

In [ ]:
z=0

TctlList=getCtlList('T')
Tds=xgrads.open_mfdataset(TctlList)
Tz=Tds.T[:,z,:,:]
Tzmean=getVarTimeMean('T')[z,:,:]

VctlList=getCtlList('V')
Vds=xgrads.open_mfdataset(VctlList)
Vz=Vds.V[:,z,:,:]
Vzmean=getVarTimeMean('V')[z,:,:]

Tzprime=Tz-Tzmean
Vzprime=Vz-Vzmean

Tzprime=Tzprime.chunk('auto')
Vzprime=Vzprime.chunk('auto')
print(Tzprime)
print(Vzprime)

In [ ]:
VT=(Tzprime*Vzprime).mean(dim='time')
VT

In [ ]:
VT.compute()

In [ ]:
VTcomp.to_netcdf('VT.nc')

In [ ]:
VT=xa.open_dataarray('VT.nc')
VT

In [ ]:
VT.plot()